In [2]:
import sys,random,math
from collections import Counter
import numpy as np

np.random.seed(1)
random.seed(1)
f = open('data/reviews.txt')
raw_reviews = f.readlines()
f.close()

tokens = list(map(lambda x:(x.split(" ")),raw_reviews))
wordcnt = Counter()
for sent in tokens:
    for word in sent:
        wordcnt[word] -= 1
vocab = list(set(map(lambda x:x[0],wordcnt.most_common())))

word2index = {}
for i,word in enumerate(vocab):
    word2index[word]=i

concatenated = list()
input_dataset = list()
for sent in tokens:
    sent_indices = list()
    for word in sent:
        try:
            sent_indices.append(word2index[word])
            concatenated.append(word2index[word])
        except:
            ""
    input_dataset.append(sent_indices)
concatenated = np.array(concatenated)
random.shuffle(input_dataset)

In [20]:
#wordcnt.most_common()
#'the' in vocab
print(len(concatenated))
print(len(input_dataset))

7459318
25000


In [ ]:
#vocab is a list with all the unique words
print(f'Vocab lenght: {len(vocab)}')
# word2index gets the index for a word
print(f'Index of word the: {word2index["the"]}')
# input_dataset is the reviews but instead of words we have the word indexes    
print(f'First review with word indexes: {input_dataset[0]}')

In [21]:
alpha, iterations = (0.05, 2)
hidden_size,window,negative = (50,2,5)

weights_0_1 = (np.random.rand(len(vocab),hidden_size) - 0.5) * 0.2
weights_1_2 = np.random.rand(len(vocab),hidden_size)*0

layer_2_target = np.zeros(negative+1)
layer_2_target[0] = 1

def similar(target='beautiful'):
  target_index = word2index[target]

  scores = Counter()
  for word,index in word2index.items():
    raw_difference = weights_0_1[index] - (weights_0_1[target_index])
    squared_difference = raw_difference * raw_difference
    scores[word] = -math.sqrt(sum(squared_difference))
  return scores.most_common(10)

def sigmoid(x):
    return 1/(1 + np.exp(-x))

for rev_i,review in enumerate(input_dataset * iterations):
  for target_i in range(len(review)):
        
    # since it's really expensive to predict every vocabulary
    # we're only going to predict a random subset
    target_samples = [review[target_i]]+list(concatenated\
    [(np.random.rand(negative)*len(concatenated)).astype('int').tolist()])

    left_context = review[max(0,target_i-window):target_i]
    right_context = review[target_i+1:min(len(review),target_i+window)]

    layer_1 = np.mean(weights_0_1[left_context+right_context],axis=0)
    layer_2 = sigmoid(layer_1.dot(weights_1_2[target_samples].T))
    layer_2_delta = layer_2 - layer_2_target
    layer_1_delta = layer_2_delta.dot(weights_1_2[target_samples])

    weights_0_1[left_context+right_context] -= layer_1_delta * alpha
    weights_1_2[target_samples] -= np.outer(layer_2_delta,layer_1)*alpha

  if(rev_i % 250 == 0):
    sys.stdout.write('\rProgress:'+str(rev_i/float(len(input_dataset)
        *iterations)) + "   " + str(similar('terrible')))
  sys.stdout.write('\rProgress:'+str(rev_i/float(len(input_dataset)
        *iterations)))

Progress:0.99998 [('terrible', -0.0), ('horrible', -2.887466226536251), ('brilliant', -2.999793458339704), ('superb', -3.6125945575213896), ('phenomenal', -3.619544252760703), ('masterful', -3.6791543901580654), ('pathetic', -3.6884210242343696), ('marvelous', -3.785183662325907), ('dreadful', -3.971738394347782), ('terrific', -4.001907892012459)]]0653)]]105)][('terrible', -0.0), ('horrible', -2.793787985794443), ('brilliant', -3.272626810081273), ('pathetic', -3.641081005223698), ('phenomenal', -3.7102952158431663), ('masterful', -3.8276731963078157), ('bad', -3.859150140548132), ('superb', -3.8707962287253297), ('marvelous', -3.9114104472263094), ('dreadful', -4.026023199643079)]


In [22]:
print(similar('terrible'))

[('terrible', -0.0), ('horrible', -2.793787985794443), ('brilliant', -3.272626810081273), ('pathetic', -3.641081005223698), ('phenomenal', -3.7102952158431663), ('masterful', -3.8276731963078157), ('bad', -3.859150140548132), ('superb', -3.8707962287253297), ('marvelous', -3.9114104472263094), ('dreadful', -4.026023199643079)]
